## 보행어린이 사고다발지역정보 API
- 반경 200m에서 12세 이하 보행어린이사고가 3건이상 또는 사망사고 2건이상 지점으로 , 경찰청 최종 선정 결과(안전을 위해 시설개선이 필요한 지역 자체선정등)을 통해 선정됩니다. 이러한 보행 어린이사고에 대한 다발지역정보를 선택한 조건에 따라 json/xml형식으로 제공합니다.
- 데이터 원본: https://taas.koroad.or.kr/api/selectChildDataSet.do

In [38]:
import pandas as pd
import numpy as np
import folium        # 지도 라이브러리
import matplotlib.pyplot as plt

from util.myUtil import MySQLDao
dao = MySQLDao()

# 차트 한글깨짐 방지
import matplotlib as mp
mp.rcParams['font.family'] = 'Malgun Gothic'
mp.rcParams['axes.unicode_minus'] = False

### A01. 보행어린이 사고다발지역 정보 > 테이블 생성하기

In [28]:
# 1.1. 데이터 전체 삭제
dao.execute("drop table chl_acdt_info")

# 1.2. 테이블 생성
dao.execute("""
CREATE TABLE CHL_ACDT_INFO(
	ACDT_ID INT UNSIGNED NOT NULL AUTO_INCREMENT COMMENT '사고_ID',
	AFOS_FID VARCHAR(22) NOT NULL     COMMENT '다발지역FID',
	AFOS_ID VARCHAR(7) NOT NULL       COMMENT '다발지역ID',
	BJD_CD VARCHAR(10) NOT NULL       COMMENT '법정동코드',
	SPOT_CD VARCHAR(10) NOT NULL      COMMENT '지점코드',
	SIDO_SGG_NM VARCHAR(40) NOT NULL  COMMENT '시도시군구명',
	SPOT_NM VARCHAR(100) NOT NULL     COMMENT '지점명',
	OCCRRNC_CNT INT UNSIGNED          COMMENT '사고건수',
	CASLT_CNT INT UNSIGNED            COMMENT '사상자수',
	DTH_DNV_CNT INT UNSIGNED NOT NULL COMMENT '사망자수',
	SE_DNV_CNT INT UNSIGNED NOT NULL  COMMENT '중상자수',
	SL_DNV_CNT INT UNSIGNED NOT NULL  COMMENT '경상자수',
	WND_DNV_CNT INT UNSIGNED NOT NULL COMMENT '부상신고자수',
	LO_CRD VARCHAR(16) NOT NULL       COMMENT '경도',
	LA_CRD	VARCHAR(15) NOT NULL      COMMENT '위도',
	GEOM_JSON TEXT NOT NULL           COMMENT '다발지역폴리곤',
	PRIMARY KEY(ACDT_ID)
) COMMENT='보행어린이_사고다발지정보'
""")

# 1.3. 데이터 등록

0

### A02. 보행어린이 사고다발지역 정보 > CSV 파일을 dataframe 으로 변환

In [36]:
df = pd.read_csv("data/01_02_보행어린이_사고다발지역정보.csv", encoding='cp949')
# df.columns = items
df.head(2)

,사고다발지FID,사고다발지ID,법정동코드,지점코드,시도시군구명,지점명,발생건수,사상자수,사망자수,중상자수,경상자수,부상신고자수,경도,위도,다발지역폴리곤
0,138468,2013097,1117013100,11170001,서울특별시 용산구1,서울특별시 용산구 한남동(순천향병원 부근),3,3,0,0,2,1,127.005586,37.533295,"{""type"":""Polygon"",""coordinates"":[[[127.0071863..."
1,144026,2013097,1120010700,11200001,서울특별시 성동구1,서울특별시 성동구 행당동(무학여고앞 부근),4,5,0,0,5,0,127.032724,37.558635,"{""type"":""Polygon"",""coordinates"":[[[127.034325,..."


### A03. 보행어린이 사고다발지역 정보 > 변환된 dataframe 에서 데이터를 추출하여 DB에 등록한다.

In [30]:
# 등록쿼리 생성하기
cols = 'AFOS_FID,AFOS_ID,BJD_CD,SPOT_CD,SIDO_SGG_NM,SPOT_NM,OCCRRNC_CNT,CASLT_CNT,DTH_DNV_CNT,SE_DNV_CNT,SL_DNV_CNT,WND_DNV_CNT,LO_CRD,LA_CRD,GEOM_JSON'
items = cols.split(',')
columns = []
for item in items:
    columns.append("%s")

cols_p = ",".join(columns)
sql = f"INSERT INTO chl_acdt_info({cols}) VALUES({cols_p})"

target_cnt = len(df) # 대상건수
proc_cnt = 0   # 처리건수
def save(param): # [등록 / 수정] 단건 처리
    return dao.execute(sql, param)

for v in df.values:
    proc_cnt += save(tuple(v))

# 3단계 : 처리 결과 출력
print("처리결과: ",{'대상건수':target_cnt, '처리건수':proc_cnt, '결과':(target_cnt==proc_cnt)})

처리결과:  {'대상건수': 964, '처리건수': 964, '결과': True}


### A04. 보행어린이 사고다발지역 정보 > DB에 등록된 데이터를 조회한다.

In [35]:
pd.DataFrame(dao.rows("select * from chl_acdt_info")).head(2)

,ACDT_ID,AFOS_FID,AFOS_ID,BJD_CD,SPOT_CD,SIDO_SGG_NM,SPOT_NM,OCCRRNC_CNT,CASLT_CNT,DTH_DNV_CNT,SE_DNV_CNT,SL_DNV_CNT,WND_DNV_CNT,LO_CRD,LA_CRD,GEOM_JSON
0,1,138468,2013097,1117013100,11170001,서울특별시 용산구1,서울특별시 용산구 한남동(순천향병원 부근),3,3,0,0,2,1,127.0055862,37.5332951,"{""type"":""Polygon"",""coordinates"":[[[127.0071863..."
1,2,144026,2013097,1120010700,11200001,서울특별시 성동구1,서울특별시 성동구 행당동(무학여고앞 부근),4,5,0,0,5,0,127.0327244,37.55863534,"{""type"":""Polygon"",""coordinates"":[[[127.034325,..."


In [120]:
df1 = pd.DataFrame(dao.rows("select * from chl_acdt_info")).head(1) # 데이터 1번째만 표시
geom = eval(df1.GEOM_JSON.to_dict().get(0))['coordinates'][0]
x = df1.LA_CRD # 위도
y = df1.LO_CRD # 경도
m = folium.Map( location=(x, y) , zoom_start=17) # 지도
for g in geom:
    folium.Marker( location=(g[1], g[0]), popup=df1['SPOT_NM'] ).add_to(m) # 지도에 마커 추가

display(m)